In [1]:
import pandas as pd
from joblib import dump
import uvicorn
from fastapi import FastAPI

import numpy as np
import pickle
import pandas as pd


import sys
sys.path.insert(0, '../fraud-detector/')
sys.path.insert(0, '../data')
sys.path.insert(0, '../')

import conf.scoring_conf as conf
from utils import load_data_local
from feateng import FeatPipeline
import joblib

###############################
# 0. Variable Assignment
###############################

inpath = conf.toscore_inpath # input path of data to be scored
intype = conf.toscore_intype # input path type of data to be scored
model_chosen = conf.model_chosen

###############################
# 1. Data read
###############################
df = load_data_local(inpath, intype)

###############################
# 2. Feature engineering pipeline
###############################
df2 = FeatPipeline.fit_transform(df)

###############################
# 3. Scoring
###############################
model = joblib.load(model_chosen+ ".pkl")

# Convert data to json
target_feature = ["isFraud"]
id_features = ['nameDest', 'nameOrig']
all_features = df.columns.difference(target_feature).difference(id_features)
data = pd.DataFrame.to_json(df2[all_features])

from pydantic import BaseModel

# TODO: make configurable
class Transaction(BaseModel):
    amount: float
    isDestBalanceNewZero: int
    isDestBalanceOldZero: int
    isDestMerchant: int
    isOrigBalanceNewZero : int
    isOrigBalanceOldZero : int
    newbalanceDest : float
    newbalanceOrig : float
    oldbalanceDest : float
    oldbalanceOrg : float
    step: int
    stepDay: int
    stepHour: int
    stepWeekDay: int
    type: str


In [2]:
df2[all_features].head()

,amount,isDestBalanceNewZero,isDestBalanceOldZero,isDestMerchant,isFlaggedFraud,isOrigBalanceNewZero,isOrigBalanceOldZero,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,stepDay,stepHour,stepWeekDay,type
0,309128.55,0,0,0,0,1,1,743261.26,0.0,434132.72,0.00,321,14.0,9.0,0.0,CASH_OUT
1,1035588.40,0,1,0,0,1,0,1035588.40,0.0,0.00,25691.00,154,7.0,10.0,0.0,TRANSFER
2,320499.65,0,0,0,0,0,0,85338.15,8646714.8,405837.80,8326215.15,348,15.0,12.0,1.0,CASH_IN
3,272780.92,0,0,0,0,1,1,928395.54,0.0,655614.62,0.00,262,11.0,22.0,4.0,TRANSFER
4,11310.62,0,0,0,0,1,1,1068365.73,0.0,1057055.11,0.00,227,10.0,11.0,3.0,CASH_OUT


In [28]:
d = [{
  "amount": 309128.55,
  "isDestBalanceNewZero": 0,
  "isDestBalanceOldZero": 0,
  "isDestMerchant": 0,
  "isOrigBalanceNewZero": 1,
  "isOrigBalanceOldZero": 1,
  "newbalanceDest": 743261.26,
  "newbalanceOrig": 0.0,
  "oldbalanceDest": 434132.72,
  "oldbalanceOrg": 0.00,
  "step": 321,
  "stepDay": 14.0,
  "stepHour": 9.0,
  "stepWeekDay": 0.0,
  "type": "CASH_OUT"
},
{
  "amount": 309128.55,
  "isDestBalanceNewZero": 0,
  "isDestBalanceOldZero": 0,
  "isDestMerchant": 0,
  "isOrigBalanceNewZero": 1,
  "isOrigBalanceOldZero": 1,
  "newbalanceDest": 743261.26,
  "newbalanceOrig": 0.0,
  "oldbalanceDest": 434132.72,
  "oldbalanceOrg": 0.00,
  "step": 321,
  "stepDay": 14.0,
  "stepHour": 9.0,
  "stepWeekDay": 0.0,
  "type": "TRANSFER"
}]

In [34]:
pd.DataFrame(d)

,amount,isDestBalanceNewZero,isDestBalanceOldZero,isDestMerchant,isOrigBalanceNewZero,isOrigBalanceOldZero,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,stepDay,stepHour,stepWeekDay,type
0,309128.55,0,0,0,1,1,743261.26,0.0,434132.72,0.0,321,14.0,9.0,0.0,CASH_OUT
1,309128.55,0,0,0,1,1,743261.26,0.0,434132.72,0.0,321,14.0,9.0,0.0,TRANSFER


In [37]:
df = pd.DataFrame(d)
df["apiTagged"] = ""
df["apiTagged"][0] = True
df

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,amount,isDestBalanceNewZero,isDestBalanceOldZero,isDestMerchant,isOrigBalanceNewZero,isOrigBalanceOldZero,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,stepDay,stepHour,stepWeekDay,type,apiTagged
0,309128.55,0,0,0,1,1,743261.26,0.0,434132.72,0.0,321,14.0,9.0,0.0,CASH_OUT,True
1,309128.55,0,0,0,1,1,743261.26,0.0,434132.72,0.0,321,14.0,9.0,0.0,TRANSFER,


In [51]:
from datetime import datetime
# dd/mm/YY
now = datetime.now()
now.strftime("%d-%m-%Y-%H-%M-%S")

'12-06-2021-16-19-48'

In [11]:
model.predict(df2[:3][all_features])

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


array([0, 0, 0])

In [18]:
data = pd.DataFrame.to_json(df2[:2][all_features])
data

'{"amount":{"0":309128.55,"1":1035588.4},"isDestBalanceNewZero":{"0":0,"1":0},"isDestBalanceOldZero":{"0":0,"1":1},"isDestMerchant":{"0":0,"1":0},"isFlaggedFraud":{"0":0,"1":0},"isOrigBalanceNewZero":{"0":1,"1":1},"isOrigBalanceOldZero":{"0":1,"1":0},"newbalanceDest":{"0":743261.26,"1":1035588.4},"newbalanceOrig":{"0":0.0,"1":0.0},"oldbalanceDest":{"0":434132.72,"1":0.0},"oldbalanceOrg":{"0":0.0,"1":25691.0},"step":{"0":321,"1":154},"stepDay":{"0":14.0,"1":7.0},"stepHour":{"0":9.0,"1":10.0},"stepWeekDay":{"0":0.0,"1":0.0},"type":{"0":"CASH_OUT","1":"TRANSFER"}}'

In [ ]:
{
    "data": [
        [309128.55,0,0,0,0,1,1,743261.26,0,434132.72,0,321,14,9,0,"CASH_OUT"],
        [309128.55,0,0,0,0,1,1,743261.26,0,434132.72,0,321,14,9,0,"TRANSFER"]
    ]
}

In [55]:
import os
files_and_directories = os.listdir("../data/")
files_and_directories



['sample_for_scoring.csv',
 'sample_to_score.txt',
 '.DS_Store',
 'input_data.csv',
 'automation']

In [68]:
import requests

def score_api(file_list):
    #curl - X
    #POST
    #"http://127.0.0.1:5000/predict" - H \
    #        "accept: application/json" \
    #        - H
    #"Content-Type: application/json" \
    #- d
    #"{\"data\":[[4.8,3,1.4,0.3],[2,1,3.2,1.1]]}"#

    url = "http://127.0.0.1:5000/predict_automation"
    headers = {'content-type': 'application/json'}
    response = requests.post(url, data=file_list, headers=headers)

    #with open("filename.xml", "w") as fd:
    #    fd.write(r

In [71]:
pd.read_csv("../data/automation_in/to_score_2021_06_13_12_15_28.csv") 

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,304,PAYMENT,12790.17,C575299289,874.00,0.00,M229858881,0.00,0.00,0,0
1,263,CASH_IN,361715.50,C393751636,5014921.78,5376637.28,C1808891905,2034588.20,1672872.70,0,0
2,8,TRANSFER,438246.69,C65655204,0.00,0.00,C2087040347,999529.08,1437775.77,0,0
3,250,TRANSFER,51280.63,C1588912202,0.00,0.00,C2016695297,1375864.81,1427145.44,0,0
4,213,PAYMENT,12817.09,C458966348,44013.00,31195.91,M471764738,0.00,0.00,0,0
5,404,PAYMENT,5851.66,C396329914,46492.00,40640.34,M1137343638,0.00,0.00,0,0
6,134,CASH_IN,9285.22,C717621671,16746379.46,16755664.68,C577022771,2962347.79,2953062.57,0,0
7,12,CASH_OUT,254414.78,C1421322613,0.00,0.00,C1040847151,13143894.15,13483456.42,0,0
8,181,PAYMENT,1635.55,C1024585201,0.00,0.00,M556121171,0.00,0.00,0,0
9,298,CASH_IN,793887.65,C1270835346,20167.00,814054.65,C981821775,0.00,0.00,0,0


In [65]:
files_earlier = [1,2,3,4]
files_now = [1,2,3,4,5,6,7]

[x for x in files_now if x not in files_earlier]

[5, 6, 7]